In [1]:
import pandas as pd
import os
import numpy as np
from datetime import date
from tqdm import tqdm
import warnings

#warnings.filterwarnings("ignore")
#%pylab inline 
#%load_ext nb_black
import seaborn as sb

In [2]:
atrasos = pd.read_csv("../dex/clean/modelo.csv", delimiter=";", low_memory=True)
atrasos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135108 entries, 0 to 2135107
Data columns (total 64 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   var_mes                     int64  
 1   dex_cliente_id              int64  
 2   saldo                       float64
 3   en_plazo                    float64
 4   atr_1_7_dias                float64
 5   atr_8_15_dias               float64
 6   atr_16_30_dias              float64
 7   atr_31_45_dias              float64
 8   atr_46_60_dias              float64
 9   atr_61_90_dias              float64
 10  atr_91_120_dias             float64
 11  atr_121_150_dias            float64
 12  atr_151_180_dias            float64
 13  atr_181_360_dias            float64
 14  atr_360_dias_a_mas          float64
 15  nuevo_tramo                 int64  
 16  fecha_obs                   object 
 17  deuda_tot_01_A              float64
 18  tramo_mora_princip_01_A     float64
 19  deuda_tot_02_A       

In [3]:
clientes = pd.read_csv("../dex/clientes.csv", delimiter=";", low_memory=True)
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796389 entries, 0 to 796388
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       796389 non-null  int64 
 1   name     796389 non-null  object
 2   type     796389 non-null  object
 3   channel  796389 non-null  object
dtypes: int64(1), object(3)
memory usage: 24.3+ MB


In [4]:
df = atrasos.merge(clientes, left_on="dex_cliente_id", right_on="id", how="left")
df.head()

,var_mes,dex_cliente_id,saldo,en_plazo,atr_1_7_dias,atr_8_15_dias,atr_16_30_dias,atr_31_45_dias,atr_46_60_dias,atr_61_90_dias,...,mora_04_d,mora_05_d,mora_06_d,num_meses_con_deuda_6_post,flag_default,tmp_max_6m,id,name,type,channel
0,201701,12002,555.47,0.00,0.0,0.00,555.47,0.0,0.0,0.0,...,0.0,0.0,0.0,6,0,NaN,12002.0,MARIA GARCIA SACCACO DE PILLACA,PUESTO DE MERCADO,MINORISTAS
1,201701,12045,3083.26,3083.26,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,6,0,NaN,12045.0,HILDA MORALES ROJAS,PUESTO DE MERCADO,MINORISTAS
2,201701,12050,168.51,168.51,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,6,0,NaN,12050.0,BERTHA BARRIENTOS ROCA,PUESTO DE MERCADO,MINORISTAS
3,201701,12054,927.16,550.97,0.0,376.19,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,6,0,NaN,12054.0,ESPERANZA MAYORCA DE LA CRUZ,PUESTO DE MERCADO,MINORISTAS
4,201701,12081,1129.48,999.87,0.0,0.00,129.61,0.0,0.0,0.0,...,0.0,0.0,0.0,6,0,NaN,12081.0,SUSANA ZEVALLOS VILLACORTA,BODEGAS,MINORISTAS


In [5]:
df = df.drop(["id"], axis=1)
df = df[
    [
        "var_mes",
        "dex_cliente_id",
        "name",
        "type",
        "channel",
        "nuevo_tramo",
        "flag_default",
        "saldo",
        "en_plazo",
        "atr_1_7_dias",
        "atr_8_15_dias",
        "atr_16_30_dias",
        "atr_31_45_dias",
        "atr_46_60_dias",
        "atr_61_90_dias",
        "atr_91_120_dias",
        "atr_121_150_dias",
        "atr_151_180_dias",
        "atr_181_360_dias",
        "atr_360_dias_a_mas",
    ]
]
df = df.rename(columns={"nuevo_tramo": "tramo"})

df.head()

,var_mes,dex_cliente_id,name,type,channel,tramo,flag_default,saldo,en_plazo,atr_1_7_dias,atr_8_15_dias,atr_16_30_dias,atr_31_45_dias,atr_46_60_dias,atr_61_90_dias,atr_91_120_dias,atr_121_150_dias,atr_151_180_dias,atr_181_360_dias,atr_360_dias_a_mas
0,201701,12002,MARIA GARCIA SACCACO DE PILLACA,PUESTO DE MERCADO,MINORISTAS,3,0,555.47,0.00,0.0,0.00,555.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,201701,12045,HILDA MORALES ROJAS,PUESTO DE MERCADO,MINORISTAS,0,0,3083.26,3083.26,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,201701,12050,BERTHA BARRIENTOS ROCA,PUESTO DE MERCADO,MINORISTAS,0,0,168.51,168.51,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,201701,12054,ESPERANZA MAYORCA DE LA CRUZ,PUESTO DE MERCADO,MINORISTAS,2,0,927.16,550.97,0.0,376.19,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,201701,12081,SUSANA ZEVALLOS VILLACORTA,BODEGAS,MINORISTAS,0,0,1129.48,999.87,0.0,0.00,129.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df = df[df["type"].notna()]

In [7]:
df = df[
    df["type"].isin(
        ["BODEGAS", "PUESTO DE MERCADO", "GASTRONOMÍA", "PANIFICACIÓN", "MAYORISTAS"]
    )
]

In [8]:
df = df[df["var_mes"].between(201801, 202012)]

df["var_mes"].unique()

array([201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808,
       201809, 201810, 201811, 201812, 201901, 201902, 201903, 201904,
       201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912,
       202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008,
       202009, 202010, 202011, 202012])

In [9]:
df[["dex_cliente_id"]] = df["dex_cliente_id"].astype("int64")
df.iloc[:, 7:] = df.iloc[:, 7:].fillna(0)

In [10]:
df["deuda_default"] = (
    df["atr_61_90_dias"]
    + df["atr_91_120_dias"]
    + df["atr_121_150_dias"]
    + df["atr_151_180_dias"]
    + df["atr_181_360_dias"]
    + df["atr_360_dias_a_mas"]
)

In [11]:
df["tipo_dex"] = np.where(
    (df["type"] == "GASTRONOMÍA") | (df["type"] == "PANIFICACIÓN"), "B2B", "CMP"
)
# df['flag_default'] = np.where(df['deuda_default'] > 0,1,0)

df["year"] = df["var_mes"].astype(str).str.slice(stop=4)

In [12]:
df.head()

,var_mes,dex_cliente_id,name,type,channel,tramo,flag_default,saldo,en_plazo,atr_1_7_dias,...,atr_46_60_dias,atr_61_90_dias,atr_91_120_dias,atr_121_150_dias,atr_151_180_dias,atr_181_360_dias,atr_360_dias_a_mas,deuda_default,tipo_dex,year
594696,201801,12002,MARIA GARCIA SACCACO DE PILLACA,PUESTO DE MERCADO,MINORISTAS,1,0,246.83,0.00,246.83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CMP,2018
594697,201801,12009,GUILLERMINA QUINTANA HUACHACA,BODEGAS,MINORISTAS,0,0,228.81,228.81,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CMP,2018
594698,201801,12017,TEOFILO VILLA HUAYLAS,PUESTO DE MERCADO,MINORISTAS,2,0,274.33,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CMP,2018
594699,201801,12045,HILDA MORALES ROJAS,PUESTO DE MERCADO,MINORISTAS,2,0,1305.37,731.48,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CMP,2018
594700,201801,12050,BERTHA BARRIENTOS ROCA,PUESTO DE MERCADO,MINORISTAS,0,0,183.86,183.86,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CMP,2018


In [13]:
df.var_mes.unique()

array([201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808,
       201809, 201810, 201811, 201812, 201901, 201902, 201903, 201904,
       201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912,
       202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008,
       202009, 202010, 202011, 202012])

In [14]:
var_mes = [
    201811,
    201812,
    201901,
    201902,
    201903,
    201904,
    201905,
    201906,
    201907,
    201908,
    201909,
    201910,
    201911,
    201912,
    202001,
    202002,
    202003,
    202004,
    202005,
    202006,
    202007,
    202008,
    202009,
    202010,
    202011,
    202012,
]

In [15]:
temporal3 = pd.DataFrame()
for i in range(1, len(var_mes) - 6):
    temporal = df[df.var_mes == var_mes[i]].copy()
    for j in [-1, 1, 2, 3, 4, 5, 6]:
        temporal2 = df[df.var_mes == var_mes[i + j]][["dex_cliente_id", "tramo"]].copy()
        temporal2.columns = ["dex_cliente_id", "tramo_" + str(j)]
        temporal = pd.merge(temporal, temporal2, how="left", on="dex_cliente_id")

    temporal3 = pd.concat([temporal3, temporal], axis=0)
    print(temporal3.shape)

(46058, 30)
(88161, 30)
(130153, 30)
(172162, 30)
(214523, 30)
(272033, 30)
(330728, 30)
(388412, 30)
(445886, 30)
(490974, 30)
(538457, 30)
(588857, 30)
(636123, 30)
(679172, 30)
(721846, 30)
(758700, 30)
(784671, 30)
(804420, 30)
(819572, 30)


In [16]:
temporal3.head()

,var_mes,dex_cliente_id,name,type,channel,tramo,flag_default,saldo,en_plazo,atr_1_7_dias,...,deuda_default,tipo_dex,year,tramo_-1,tramo_1,tramo_2,tramo_3,tramo_4,tramo_5,tramo_6
0,201812,12002,MARIA GARCIA SACCACO DE PILLACA,PUESTO DE MERCADO,MINORISTAS,3,0,1033.59,0.00,0.00,...,0.0,CMP,2018,3.0,1.0,3.0,2.0,2.0,2.0,4.0
1,201812,12009,GUILLERMINA QUINTANA HUACHACA,BODEGAS,MINORISTAS,1,0,1712.03,1137.39,574.64,...,0.0,CMP,2018,0.0,0.0,1.0,2.0,0.0,0.0,0.0
2,201812,12017,TEOFILO VILLA HUAYLAS,PUESTO DE MERCADO,MINORISTAS,3,0,761.33,0.00,0.00,...,0.0,CMP,2018,0.0,1.0,3.0,0.0,3.0,2.0,0.0
3,201812,12045,HILDA MORALES ROJAS,PUESTO DE MERCADO,MINORISTAS,3,0,2086.91,0.00,0.00,...,0.0,CMP,2018,4.0,2.0,3.0,2.0,0.0,3.0,2.0
4,201812,12048,ISABEL JURO HUAYLLANRE,PUESTO DE MERCADO,MINORISTAS,2,0,446.00,0.00,0.00,...,0.0,CMP,2018,2.0,1.0,1.0,1.0,2.0,NaN,NaN


In [17]:
temporal3.columns

Index(['var_mes', 'dex_cliente_id', 'name', 'type', 'channel', 'tramo',
       'flag_default', 'saldo', 'en_plazo', 'atr_1_7_dias', 'atr_8_15_dias',
       'atr_16_30_dias', 'atr_31_45_dias', 'atr_46_60_dias', 'atr_61_90_dias',
       'atr_91_120_dias', 'atr_121_150_dias', 'atr_151_180_dias',
       'atr_181_360_dias', 'atr_360_dias_a_mas', 'deuda_default', 'tipo_dex',
       'year', 'tramo_-1', 'tramo_1', 'tramo_2', 'tramo_3', 'tramo_4',
       'tramo_5', 'tramo_6'],
      dtype='object')

In [18]:
var = ["tramo_-1", "tramo_1", "tramo_2", "tramo_3", "tramo_4", "tramo_5", "tramo_6"]

for i in var:
    temporal3[i].fillna(0, inplace=True)

In [19]:
temp = temporal3[(temporal3["tramo_-1"] < 3) & (temporal3["tramo"] >= 3)].copy()

In [20]:
temp[
    [
        "var_mes",
        "dex_cliente_id",
        "type",
        "tramo",
        "tramo_-1",
        "tramo_1",
        "tramo_2",
        "tramo_3",
        "tramo_4",
        "tramo_5",
        "tramo_6",
    ]
].to_clipboard()

In [21]:
pd.crosstab(temp.tramo, temp.tramo_1)

tramo_1,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
tramo,,,,,,,,,,,,
3,28753,10367,12121,12346,2318,12874,392,17,8,9,148,30
4,11549,3144,3684,4037,1107,747,8744,16,12,8,54,16
5,372,77,73,71,23,19,304,0,0,0,5,0
6,114,35,23,26,4,8,5,109,2,0,0,0
7,60,14,12,7,1,0,2,1,58,0,0,0
8,26,9,4,5,1,2,0,0,1,48,1,0
9,13,7,5,1,1,0,0,1,0,0,38,0
10,41,22,16,9,1,4,0,1,0,0,427,502
11,26,10,5,2,0,0,1,0,0,0,0,341


In [22]:
temp = temporal3[
    (temporal3["tramo_-1"] < 4)
    & (temporal3["tramo"] >= 4)  # & (temporal3["tramo"] <= 6)
].copy()

In [23]:
temp[
    [
        "var_mes",
        "dex_cliente_id",
        "type",
        "tramo",
        "tramo_-1",
        "tramo_1",
        "tramo_2",
        "tramo_3",
        "tramo_4",
        "tramo_5",
        "tramo_6",
    ]
].to_clipboard()

In [24]:
pd.crosstab(temp.tramo, temp.tramo_1)

tramo_1,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
tramo,,,,,,,,,,,,
4,12651,3611,4322,5196,1770,1035,9598,16,12,8,68,19
5,6566,1113,1278,1474,576,523,7810,126,4,8,61,25
6,313,70,66,53,12,19,67,285,4,0,0,0
7,71,18,14,12,2,1,5,1,71,2,0,0
8,30,10,5,6,1,2,1,0,1,62,2,0
9,15,9,7,1,1,0,1,1,0,0,47,0
10,55,28,23,15,1,5,1,1,0,1,516,630
11,34,11,7,5,0,0,1,0,0,0,2,378


In [25]:
temp = temporal3[
    (temporal3["tramo_-1"] < 5)
    & (temporal3["tramo"] >= 5)  # & (temporal3["tramo"] <= 6)
].copy()

In [26]:
temp[
    [
        "var_mes",
        "dex_cliente_id",
        "type",
        "tramo",
        "tramo_-1",
        "tramo_1",
        "tramo_2",
        "tramo_3",
        "tramo_4",
        "tramo_5",
        "tramo_6",
    ]
].to_clipboard()

In [27]:
pd.crosstab(temp.tramo, temp.tramo_1)

tramo_1,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
tramo,,,,,,,,,,,,
5,6885,1206,1390,1682,712,639,8215,158,5,8,66,26
6,4075,551,560,556,220,251,487,4963,12,1,22,7
7,79,18,14,14,4,1,5,1,87,2,0,1
8,35,10,7,6,1,2,2,0,1,71,2,0
9,17,10,8,2,1,0,1,2,0,0,52,0
10,59,30,27,17,2,5,2,2,0,1,548,668
11,34,11,7,6,1,0,2,0,0,0,2,400
